In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Texturizer Experiment

In this notebook we explore how far we can go with languages statistics, word and pattern matching features created using the Texturizer package.

Note: The model tuning code in this notebook borrows heavily from code in a [Notebook](https://www.kaggle.com/jnegrini/commonlit-readability-nlp-eda-and-initial-model) by [Juliana Negrini de Araujo](https://www.kaggle.com/jnegrini)



In [ ]:
!pip install texturizer==0.1.6

In [ ]:
df_train = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")
df_submission = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
df_train = df_train.loc[:,['excerpt','target']]
df_sub = df_submission.loc[:,['excerpt']]

In [ ]:
from texturizer.pipeline import TextTransform

### Initiaze the texturizer transformer
We add transforms for a range of text features to explore. 
This includes: simple statistics, part of speech proportions, word scarcity statistics, indicators for traits and rhetorical devices, counts of words from common topics and sentiment scores.

In [ ]:
textur  = TextTransform(columns=['excerpt'], transforms=['simple','pos','scarcity','traits','rhetoric','topics','count_matches','sentiment'])


Transform both the training data and the submission test data

In [ ]:
df_trainer = textur.fit_transform(df_train)
df_sub = textur.fit_transform(df_sub)

In [ ]:
df_trainer.head()

In [ ]:
df_trainer.columns

### Visualise Feature Relationships

Texturizer gave us a bunch of new features derived from the text. 

Lets look at how they relate to the target variable.

In [ ]:
df_trainer.corrwith(df_trainer["target"])

In [ ]:
strong_features = ['target','excerpt_len','excerpt_sc','excerpt_avg_wl','excerpt_pron','excerpt_adp','excerpt_adj','excerpt_adv']

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(20,8));
g = sns.pairplot(df_trainer,vars=strong_features,corner=True,
                 plot_kws=dict(s = 8),
                 diag_kws=dict(linewidth=0,alpha=.5));
g.fig.suptitle('Relationship between Features',fontsize=16, weight = 'bold');

In [ ]:

#ML Model
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


In [ ]:
df_trainer.head()

In [ ]:
features_train = df_trainer.drop('target', axis=1)
target_train = df_trainer['target']

In [ ]:
# Create  Train and Test sets
x_trainer, x_tester, y_trainer, y_tester = train_test_split(features_train, target_train, test_size=0.10,random_state=42)

In [ ]:
x_trainer.head()

In [ ]:
print ("Training set size", x_trainer.shape[0])
print ("Test set size",x_tester.shape[0])

#Define K-Fold Validation
kf = KFold(n_splits=10, shuffle = True, random_state = 42)

# MODEL
Build a model by first cross validating to explore hyperparameters.

In [ ]:
#Optimisation
import pickle
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING) #Disable Warnings

In [ ]:
def CV_LGBM(model, x_train, x_test):
    RMSE = []
    for train_index, test_index in kf.split(x_train):
            X_train, X_val = x_train.iloc[train_index], x_train.iloc[test_index]
            Y_train, Y_val = y_trainer.iloc[train_index], y_trainer.iloc[test_index]
            model.fit(X_train, Y_train, eval_metric='rmse', eval_set=[(X_val, Y_val)],verbose=0)
            predictions = model.predict(X_val)
            RMSE.append(np.sqrt(mean_squared_error(Y_val, predictions)))
    test_score = np.sqrt(mean_squared_error(y_tester, model.predict(x_tester)))
    mean_res = np.mean(RMSE)
    std_dev = np.std(RMSE)
    return mean_res,std_dev,test_score,model  

### Hyperparameter Optimisation

Run a job over the LGBM hyperparameters to determinal optimal settings.

In [ ]:
def create_model(trial): 
    #Parameters to be Optimised
    LGBM_alpha = trial.suggest_loguniform('LGBM_alpha', 0.0001, 1)
    LGBM_lambda = trial.suggest_loguniform('LGBM_lambda', 0.0001, 1)
    LGBM_num_leaves = trial.suggest_int('LGBM_num_leaves', 5, 1000)
    LGBM_max_depth = trial.suggest_int('LGBM_max_depth', 5, 1000)
    LGBM_LR = trial.suggest_loguniform('LGBM_LR', 0.001, 0.1)
    LGBM_estimators = trial.suggest_int('LGBM_estimators', 50, 10000)
    LGBM_early_stopping = trial.suggest_int('LGBM_early_stopping', 30, 300)
    #ML Model
    model = lgbm.LGBMRegressor(objective = 'regression',
                                metric = 'rmse',
                                learning_rate = LGBM_LR, #0.019, 
                                early_stopping_round = LGBM_early_stopping, #100
                                n_estimators = LGBM_estimators, #10000,
                                max_depth=LGBM_max_depth, 
                                num_leaves=LGBM_num_leaves,
                                reg_alpha = LGBM_alpha,
                                reg_lambda = LGBM_lambda,
                                verbose = -1)  
    
    if trial.should_prune(): raise optuna.TrialPruned()
    return model

def objective(trial):
    #Create and try the ML model
    model = create_model(trial)
    #Compute accuracy
    accuracy,_,_,_ = CV_LGBM(model, x_trainer, x_tester)
    # Save a trained model to a file.
    with open("{}.pickle".format(trial.number), "wb") as fout:
        pickle.dump(model, fout)
    return accuracy

#study = optuna.create_study(direction="minimize");
#study.optimize(objective, n_trials=300);
#print("Mean RMSE over X-Validation: %.3f"% (study.best_value))
#print("Best parameters: ", (study.best_params))

### Latest Run
The above cells execute a large hyperparameter search.

These will be commented out to prevent long run times before submissions. The output of these cells on the final run is copied in the text cell below:


Cell Output: 
```
Mean RMSE over X-Validation: 0.780
Best parameters:  {'LGBM_alpha': 0.003096361887930177, 'LGBM_lambda': 0.33060672765069316, 'LGBM_num_leaves': 5, 'LGBM_max_depth': 197, 'LGBM_LR': 0.06818604384594631, 'LGBM_estimators': 6464, 'LGBM_early_stopping': 180}
```

## Previous Run - From earlier code versions

Mean RMSE over X-Validation: 0.781
Best parameters:  {'LGBM_alpha': 0.000338008181838662, 'LGBM_lambda': 0.012790929119777477, 'LGBM_num_leaves': 14, 'LGBM_max_depth': 492, 'LGBM_LR': 0.041984027432934806, 'LGBM_estimators': 7822}

In [ ]:
final_LGBM = lgbm.LGBMRegressor(objective='regression',
                               metric = 'rmse',
                               verbose=-1, 
                               learning_rate=0.068, 
                               max_depth=197, 
                               num_leaves=5, 
                               early_stopping_round = 180, 
                               n_estimators = 6464,
                               reg_alpha = 0.003,
                               reg_lambda = 0.331,
                            force_col_wise=True)


In [ ]:
final_LGBM.fit(x_trainer, y_trainer, eval_metric='rmse', eval_set=[(x_tester, y_tester)],verbose=0)
test_score = np.sqrt(mean_squared_error(y_tester, final_LGBM.predict(x_tester)))
print(test_score)

In [ ]:
preds = final_LGBM.predict(x_tester)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(final_LGBM.feature_importances_,x_tester.columns)), columns=['Value','Feature'])

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

We identified a few columns that add nothing.
We remove them and retrain.

In [ ]:
remove_list = [ 'excerpt_politics', 'excerpt_sex', 'excerpt_crime', 'excerpt_cliches', 'excerpt_jargon', 'excerpt_authority']

In [ ]:
x_trainer2 = x_trainer.drop(remove_list, axis=1)
x_tester2 = x_tester.drop(remove_list, axis=1)
df_sub2 = df_sub.drop(remove_list, axis=1)
df_sub2.columns

In [ ]:
final_LGBM.fit(x_trainer2, y_trainer, eval_metric='rmse', eval_set=[(x_tester2, y_tester)],verbose=0)
test_score = np.sqrt(mean_squared_error(y_tester, final_LGBM.predict(x_tester2)))
print(test_score)

In [ ]:
df_submission['target'] = final_LGBM.predict(df_sub2)

In [ ]:
df_submission.head()

In [ ]:
df_submission[['id','target']].to_csv('submission.csv', index=False)